# 4.7: 基于生成式张量网络的概率分类方法

> Reference: _Generative tensor network classification model for supervised machine learning_

考虑利用每一类训练样本，分别训练一个GMPS，记为$|\psi^{[c]}\rangle$（其中 $c$ 代表 $c$ 类）

对于某个待分类的样本（记为$y$），各个GMPS生成该样本的概率满足玻恩概率诠释，这个概率相对于是给定了分类的条件概率，即
  
$$p(y|c) = |\langle\Phi|\psi^{[c]}\rangle|^2$$

其中 $|\Phi\rangle$ 为 $y$ 经过特征映射所得的量子态。

对分类的预测满足：$\tilde{c} = \arg\max_c p(y|c)$

这样做是合理的，例如，当 $y$ 代表一张猫的图片时，用猫的图片对应的GMPS得到该图的概率显然应大于用狗图片对应 GMPS得到该图的概率，即我们预期
  
$$p(y \in c|c) \gg p(y \not\in c|c)$$

## 证明

### 背景知识

可由贝叶斯 (Bayes) 公式给出更加严谨的数学基础：

考虑一组事件集合{$A_n$}，满足集合中任意二事件同时发生的概率为0（互斥），且所有事件发生的概率和为1，有$\sum_n p(A_n) = 1$。我们称这样的事件集合为**完备事件组**。

这组事件集合中的每一个事件都可能导致另一个事件 $B$ 的发生与否，对应的概率由条件概率 $p(B|A_n)$ 给出。此时，事件$B$发生的概率满足全概率公式

$$p(B) = \sum_n p(B|A_n)p(A_n)$$

一类简单的完备事件组是某一事件$A$及其对立事件$\bar{A}$，例如，$A$代表打雷这件事，则$\bar{A}$代表没有打雷。那么，下雨的概率满足

$$p(下雨) = p(下雨|打雷)p(打雷) + p(下雨|不打雷)p(不打雷)$$

事件$B$与某个$A_n$同时发生的概率（被称为联合概率）满足
$$p(A_n, B) = p(B|A_n)p(A_n)$$
例如，打雷且下雨的概率等于打雷的概率$p(打雷)$乘以打雷的情况下下雨的概率$p(下雨|打雷)$。

下面我们将所考虑的问题转换一下：如果我们已知打雷的概率、打雷时下雨的概率，以及不打雷时下雨地概率，能否给出下雨时打雷的概率呢？

根据联合概率公式，打雷且下雨的联合概率满足两个等式
$$p(打雷,下雨) = p(打雷|下雨)p(下雨) = p(下雨|打雷)p(打雷)$$

于是有
$$p(打雷|下雨) = \frac{p(下雨|打雷)p(打雷)}{p(下雨)}$$

其中概率$p(下雨)$并未直接给出，但可以通过全概率公式计算，即
$$p(下雨) = p(下雨|打雷)p(打雷) + p(下雨|不打雷)[1-p(打雷)]$$

另一种理论上等价的方式是利用概率的归一性，即 $p(打雷|下雨) + p(不打雷|下雨) = 1$。由于

$$p(不打雷|下雨) = \frac{p(下雨|不打雷)p(不打雷)}{p(下雨)}$$

我们只需计算 $p'(打雷|下雨) = p(下雨|打雷)p(打雷)$，以及 $p'(不打雷|下雨) = p(下雨|不打雷)[1-p(打雷)]$，并计算归一化因子
$$Z = p'(打雷|下雨) + p'(不打雷|下雨)$$

则有$p(不打雷|下雨) = p'(不打雷|下雨)/Z$。容易看出，$Z = p(下雨)$，其就是全概率公式。

综上有

$$p(打雷|下雨) = \frac{p(下雨|打雷)p(打雷)}{p(下雨|打雷)p(打雷) + p(下雨|不打雷)[1-p(打雷)]}$$

上式的一般形式为

$$p(A_n|B) = \frac{p(B|A_n)p(A_n)}{\sum_m p(B|A_m)p(A_m)}$$

该式被称为贝叶斯公式或贝叶斯定理，其中边缘（marginal）概率 $p(A_n)$ 被称为**先验（prior）概率**，条件概率 $p(A_n|B)$ 也被称为**后验（posterior）概率**，条件概率 $p(B|A_n)$ 也被称为**似然（likelihood）函数**

### GMPSC

回到基于GMPS的分类方法：GMPS给出贝叶斯公式中的似然函数 $p(y|c) = |\langle\Phi|\psi^{[c]}\rangle|^2$

先验概率 $p(c)$ 可认为是随机从训练集抽取一个样本时，所选样本属于第 $c$ 类的概率。一般情况下，训练集各类所含的样本数相等，因此我们可以取 $p(c) = 1/C$，其中 $C$ 为总类别数，因此先验概率的取值与样本 $y$ 及类别 $c$ 无关。

将上述先验概率与似然函数代入贝叶斯公式，得到样本$y$属于各类的条件（后验）概率，满足

$$p(c|y) = \frac{p(y|c)}{CZ}$$

其中 $Z$ 为归一化系数，由于类别预测满足 $\hat{c} = \text{argmax}_c p(c|y)$，通过该式不难看出，$Z$ 的值并不影响 $\hat{c}$，因此可直接取

$$\hat{c} = \text{argmax}_c p(y|c) = \text{argmax}_c |\langle\Phi|\psi^{[c]}\rangle|^2$$

依据上式实现的GMPS分类模型，被称为GMPS分类器（GMPSC）。

也可取 $\hat{c} = \text{argmax}_c \ln|\langle\Phi|\psi^{[c]}\rangle|^2$，避免计算指数小的 $|\langle\Phi|\psi^{[c]}\rangle|^2$（正交灾难）。

## Code

In [ ]:
# TODO
import torch